In [10]:
from osgeo import gdal, gdalconst, ogr, osr
import numpy as np
import rasterio
import os, glob
from tqdm import tqdm
from mosaic import mosaic1

In [11]:

def get_min_max_img_follow(img_for_follow, numband):
    """Ham nay de tinh percentile cua anh duoc chon de follow"""
    dataset = gdal.Open(img_for_follow, gdal.GA_ReadOnly)
    dict_band_min_max_percentile_follow = {k: dict(max=0, min=0) for k in range(numband)}

    for i_chan in range(numband):
        band = dataset.GetRasterBand(i_chan+1).ReadAsArray().astype(np.float16)
        band[band==0] = np.nan
        dict_band_min_max_percentile_follow[i_chan]['max'] = np.nanpercentile(band, 98)
        dict_band_min_max_percentile_follow[i_chan]['min'] = np.nanpercentile(band, 2)
    return dict_band_min_max_percentile_follow


def convert_percentile_all_file_in_dir(dir_img, numband, dict_band_min_max_percentile_follow, out_dir):
    list_fp = glob.glob(os.path.join(dir_img,'*.tif'))
    k = 0
    for image_path in tqdm(list_fp):
        # image_path = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/20220330_030416_80_248c_3B_AnalyticMS_SR_8b_clip.tif"
        dataset_ori = gdal.Open(image_path, gdal.GA_ReadOnly)
        dict_band_min_max_percentile_ori = {k: dict(max=0, min=0) for k in range(numband)}

        for i_chan in range(numband):
            band = dataset_ori.GetRasterBand(i_chan+1).ReadAsArray().astype(np.float16)
            band[band==0] = np.nan
            dict_band_min_max_percentile_ori[i_chan]['max'] = np.nanpercentile(band, 98)
            dict_band_min_max_percentile_ori[i_chan]['min'] = np.nanpercentile(band, 2)
        src = rasterio.open(image_path)
        meta = src.meta

        img_ = src.read()
        img_percentile = np.empty(img_.shape)
        for i in range(numband):
            idxnodata = np.where(img_[i]==meta['nodata'])
            # img_[i][img_[i] > dict_band_min_max_percentile[i]['max']] = dict_band_min_max_percentile[i]['max']
            # img_[i][img_[i] < dict_band_min_max_percentile[i]['min']] = dict_band_min_max_percentile[i]['min']
            # img_[i][idxnodata]=meta['nodata']

            img_percentile[i] = np.interp(img_[i], (dict_band_min_max_percentile_ori[i]['min'], dict_band_min_max_percentile_ori[i]['max']), 
                                                (dict_band_min_max_percentile_follow[i]['min'], dict_band_min_max_percentile_follow[i]['max']))
            img_percentile[i][idxnodata]=meta['nodata']
            img_percentile[i].astype('uint16')
        img_percentile = img_percentile.astype('uint16')
        # out = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/mosaic/mosaic3.tif"
        k+=1
        out = os.path.join(out_dir, f"mosaic_{k}.tif")
        with rasterio.open(out, 'w', **meta) as dst:
            dst.write(img_percentile)


## Chay ra folder can percentile

In [12]:
img_for_follow = r"/home/skm/SKM16/IMAGE/npark/npark-backend-v2/data_projects/Green Cover Npark Singapore/tmp/T7-2022/data_genorator_01/predict_float/cloud/S2B_MSIL2A_20220709T031529_N0400_R118_T48NUG_20220709T061854_haze_4_0.tif"
dir_path_need_follow = r"/home/skm/SKM16/IMAGE/npark/npark-backend-v2/data_projects/Green Cover Npark Singapore/tmp/T7-2022/data_genorator_01/predict_float/cloud"
numband = 4
out_dir_percentile = r"/home/skm/SKM16/Tmp/per_mosaic_t7_v2"
if not os.path.exists(out_dir_percentile):
    os.makedirs(out_dir_percentile)
dict_band_min_max_percentile_follow = get_min_max_img_follow(img_for_follow, numband)
convert_percentile_all_file_in_dir(dir_path_need_follow, numband, dict_band_min_max_percentile_follow, out_dir_percentile)


100%|██████████| 4/4 [00:46<00:00, 11.51s/it]


## Mosaic tuy bien hoac chon


    Trong list MOSAIC danh sach thi "O DAU LIST se la anh TREN CUNG"


In [13]:
list_img_name = [   "mosaic_1.tif",
                    "mosaic_3.tif",
                    "mosaic_4.tif",
                    "mosaic_2.tif"
]
dir_mosaic = r"/home/skm/SKM16/Tmp/per_mosaic_t7/b"
out_path = os.path.join(dir_mosaic,"mosaic1.tif")
if not os.path.exists(dir_mosaic):
    os.makedirs(dir_mosaic)
mosaic1(out_dir_percentile, list_img_name, out_path)

    Mosaic tu dong

In [ ]:

dir_mosaic = r"/home/skm/SKM16/Tmp/per_mosaic_t7/b"
